In [1]:
import aoc
import cmath
import collections
from collections import defaultdict
from functools import cache, reduce
import itertools as it
import math
import more_itertools as mit
import networkx as nx
import numpy as np
import operator as op
import pandas as pd
import re
import statistics
import utils

In [3]:
test_input="""#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########"""

# Part 1

#############
#...........#
###A#C#B#B###
  #D#D#A#C#
  #########

a3
b6
b6
a5

#############
#AB.B.....A.#
###.#C#.#.###
  #D#D#.#C#
  #########

c5
c5
d8

#############
#.B.B.....AA#
###.#.#C#.###
  #D#.#C#D#
  #########

b3
b4
d10
a9
a3


#############
#01.2.3.4.56#
###7#9#b#d###
  #8#a#c#e#
  #########

In [55]:
# run to solve part 1

import heapq

g = nx.Graph()
g.add_edge(0, 1)
g.add_edge(1, 101)
g.add_edge(101, 2)
g.add_edge(2, 102)
g.add_edge(102, 3)
g.add_edge(3, 103)
g.add_edge(103, 4)
g.add_edge(4, 104)
g.add_edge(104, 5)
g.add_edge(5, 6)
g.add_edge(101, 7)
g.add_edge(7, 8)
g.add_edge(102, 9)
g.add_edge(9, 10)
g.add_edge(103, 11)
g.add_edge(11, 12)
g.add_edge(104, 13)
g.add_edge(13, 14)

left_index = 0
right_index = 6

def is_hallway(node):
    return node < 7

def is_doorway(node):
    return node >= 100

def room_label(node):
    match node:
        case 7 | 8:
            return 'A'
        case 9 | 10:
            return 'B'
        case 11 | 12:
            return 'C'
        case 13 | 14:
            return 'D'
        case _:
            return None

desired_indices = {
    'A': 8,
    'B': 10,
    'C': 12,
    'D': 14
}

costs = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000
}

def get_goal_index(state, index):
    am = state[index]
    if am is None:
        return None
    goals = desired_indices[am]
    goal_ams = (state[i] for i in goals)
    if any(gam != am for gam in goal_ams):
        # room has a non matching am
        return None
    for g, gam in zip(goals, goal_ams):
        if gam is None:
            return g

def get_valid_moves_for_index(state, index):
    am = state[index]
    if am is None:
        return []
    
    # first check if the item can move directly to its goal
    goal = desired_indices[am]
    if index == goal:
        # already at goal
        return []
    if state[goal] == am:
        goal -= 1
    if index == goal:
        # already at goal
        return []
    if state[goal] is None:
        # allowed to path to goal
        p = nx.shortest_path(g, index, goal)
        if all(state[n] is None for n in p[1:-1] if not is_doorway(n)):
            # path clear, move to goal
            return [(goal, costs[am] * (len(p) - 1))]
    
    if is_hallway(index):
        # if index in hallway, can only move to goal from now on
        return []
    
    valid_stops = []
    p1 = nx.shortest_path(g, index, left_index)
    p2 = nx.shortest_path(g, index, right_index)
    for p in (p1, p2):
        for steps, n in enumerate(p[1:], 1):
            if is_doorway(n):
                # can't stop at doorways
                continue
            if state[n] is not None:
                # something in the way, stop
                break
            if not is_hallway(n):
                # can only stop in hallway
                continue
            valid_stops.append((n, costs[am] * steps))
    return valid_stops

def is_complete(state):
    for am, goal in desired_indices.items():
        if state[goal] != am or state[goal-1] != am:
            return False
    return True

def min_possible_score(state):
    costs = []
    for i in range(len(state) - 1):
        am = state[i]
        if am is not None:
            goal = desired_indices[am]
            p = nx.shortest_path(g, i, goal)
            costs.append((len(p) - 1) * costs[am])
    return sum(costs) - 1111

def part_1(aoc_input):
    m = re.findall(r"(\w)", aoc_input)
    game_tree = nx.DiGraph()
    print(m)
    state = None, None, None, None, None, None, None, m[0], m[4], m[1], m[5], m[2], m[6], m[3], m[7], 0
    
    #game_tree.add_node(state)
    
    best_score = 999999999
    best_state = None
    node_queue = [(0, state)]
    
    while node_queue:
        cur_state = node_queue.pop()
        if cur_state[-1] >= best_score:
            continue
        for i in range(len(cur_state) - 1):
            moves = get_valid_moves_for_index(cur_state, i)
            for mi, mc in moves:
                new_state = list(cur_state)
                new_state[i] = None
                new_state[mi] = cur_state[i]
                new_state[-1] = cur_state[-1] + mc
                new_state = tuple(new_state)
                #game_tree.add_edge(cur_state, new_state)
                if is_complete(new_state):
                    if new_state[-1] < best_score:
                        print("NEW BEST: ", new_state)
                        best_score = new_state[-1]
                        best_state = new_state
                elif new_state[-1] < best_score:
                    node_queue.append(new_state)
    #print(len(game_tree))
    return best_score

part_1_test_answer = part_1(test_input)
print("TEST RESULT: ", part_1_test_answer)
part_1_answer = part_1(aoc.get_input(2021, 23))
print("ACTUAL RESULT: ", part_1_answer)

['B', 'C', 'B', 'D', 'A', 'D', 'C', 'A']


TypeError: '>=' not supported between instances of 'tuple' and 'int'

In [30]:
# run to submit part 1

aoc.submit_answer(2021, 23, 1, part_1_answer)

Submitting for YEAR 2021, DAY 23, LEVEL 1: 18170... Correct!


# Part 2

In [65]:
# run to solve part 2

import heapq

g = nx.Graph()
g.add_edge(0, 1)
g.add_edge(1, 101)
g.add_edge(101, 2)
g.add_edge(2, 102)
g.add_edge(102, 3)
g.add_edge(3, 103)
g.add_edge(103, 4)
g.add_edge(4, 104)
g.add_edge(104, 5)
g.add_edge(5, 6)
g.add_edge(101, 7)
g.add_edge(7, 8)
g.add_edge(8, 9)
g.add_edge(9, 10)
g.add_edge(102, 11)
g.add_edge(11, 12)
g.add_edge(12, 13)
g.add_edge(13, 14)
g.add_edge(103, 15)
g.add_edge(15, 16)
g.add_edge(16, 17)
g.add_edge(17, 18)
g.add_edge(104, 19)
g.add_edge(19, 20)
g.add_edge(20, 21)
g.add_edge(21, 22)

left_index = 0
right_index = 6

def is_hallway(node):
    return node < 7

def is_doorway(node):
    return node >= 100

desired_indices = {
    'A': (10, 9, 8, 7),
    'B': (14, 13, 12, 11),
    'C': (18, 17, 16, 15),
    'D': (22, 21, 20, 19)
}

costs = {
    'A': 1,
    'B': 10,
    'C': 100,
    'D': 1000
}

@cache
def get_valid_moves_for_index(state, index):
    am = state[index]
    if am is None:
        return []
    
    # first check if the item can move directly to its goal
    goals = desired_indices[am]
    if all(state[goal] is None or state[goal] == am for goal in goals):
        for goal in goals:
            if index == goal:
                return []
            elif state[goal] is None:
                p = nx.shortest_path(g, index, goal)
                if all(state[n] is None for n in p[1:-1] if not is_doorway(n)):
                    return [(goal, costs[am] * (len(p) - 1))]
                else:
                    # path blocked
                    break
    if is_hallway(index):
        # if index in hallway, can only move to goal from now on
        return []
    
    valid_stops = []
    p1 = nx.shortest_path(g, index, left_index)
    p2 = nx.shortest_path(g, index, right_index)
    for p in (p1, p2):
        for steps, n in enumerate(p[1:], 1):
            if is_doorway(n):
                # can't stop at doorways
                continue
            if state[n] is not None:
                # something in the way, stop
                break
            if not is_hallway(n):
                # can only stop in hallway
                continue
            valid_stops.append((n, costs[am] * steps))
    return valid_stops

@cache
def is_complete(state):
    for am, goals in desired_indices.items():
        if any(state[goal] != am for goal in goals):
            return False
    return True

@cache
def min_possible_score(state):
    c = []
    for i in range(len(state)):
        am = state[i]
        if am is not None:
            goal = desired_indices[am][0]
            p = nx.shortest_path(g, i, goal)
            c.append((len(p) - 1) * costs[am])
    return sum(c) - 6666

def part_2(aoc_input):
    m = re.findall(r"(\w)", aoc_input)
    game_tree = nx.DiGraph()
    print(m)
    state = (None, None, None, None, None, None, None,
             m[0], 'D', 'D', m[4],
             m[1], 'C', 'B', m[5],
             m[2], 'B', 'A', m[6],
             m[3], 'A', 'C', m[7])
    
    #game_tree.add_node(state)
    
    best_score = 999999999
    best_state = None
    node_queue = [(0, 0, state, 0, 0)]
    
    differentiator = 0
    
    while node_queue:
        ideal_score, _, cur_state, cur_score, depth = heapq.heappop(node_queue)
        if ideal_score >= best_score or depth >= 400:
            break
        for i in range(len(cur_state)):
            moves = get_valid_moves_for_index(cur_state, i)
            for mi, mc in moves:
                new_state = list(cur_state)
                new_state[i] = None
                new_state[mi] = cur_state[i]
                new_score = cur_score + mc
                if new_score >= best_score:
                    continue
                new_state = tuple(new_state)
                #game_tree.add_edge(cur_state, new_state)
                if is_complete(new_state):
                    print("NEW BEST: ", new_score)
                    best_score = new_score
                    best_state = new_state
                else:
                    mps = min_possible_score(new_state) + new_score
                    if mps >= best_score:
                        continue
                    differentiator += 1
                    if (differentiator % 100000) == 0:
                        print(differentiator, len(node_queue))
                    heapq.heappush(node_queue, (mps, differentiator, new_state, new_score, depth + 1))
    #print(len(game_tree))
    return best_score

part_2_test_answer = part_2(test_input)
print("TEST RESULT: ", part_2_test_answer)
part_2_answer = part_2(aoc.get_input(2021, 23))
print("ACTUAL RESULT: ", part_2_answer)

['B', 'C', 'B', 'D', 'A', 'D', 'C', 'A']
100000 53140
200000 99913
300000 147445
400000 185737
500000 223518
600000 251957
700000 234969
800000 249901
900000 287878
1000000 319660
1100000 335570
1200000 356405
1300000 371568
1400000 386465
1500000 376719
1600000 382026
1700000 398642
1800000 430867
1900000 419747
2000000 411688
2100000 378828
2200000 359942
2300000 327716
2400000 344707
2500000 348159
2600000 337131
2700000 340515
2800000 362838
NEW BEST:  44169
TEST RESULT:  44169
['A', 'C', 'B', 'B', 'D', 'D', 'A', 'C']
100000 47693
200000 79482
300000 96516
400000 91690
500000 90178
600000 105150
700000 134100
800000 144446
900000 139575
1000000 126872
1100000 145907
1200000 150797
1300000 161878
1400000 172137
1500000 169242
NEW BEST:  50208
ACTUAL RESULT:  50208


In [66]:
# run to submit part 2

aoc.submit_answer(2021, 23, 2, part_2_answer)

Submitting for YEAR 2021, DAY 23, LEVEL 2: 50208... Correct!
